# Building Tools

At their core, tools are objects that consume some input, typically in the format of a string (text), and output some helpful information as a string.

In reality, they are little more than a simple function that we’d find in any code. The only difference is that tools take input from an LLM and feed their output to an LLM.

With that in mind, tools are relatively simple. Fortunately, we can build tools for our agents in no time.


## Simple Calculator Tool

We will start with a simple custom tool. The tool is a simple calculator that calculates a circle’s circumference based on the circle’s radius.


In [36]:
from langchain.tools import BaseTool
from math import pi
from typing import Union


class CircumferenceTool(BaseTool):
    name = "Circumference calculator"
    description = "use this tool when you need to calculate a circumference using the radius of a circle. The tool only accepts float or int values as input"

    def _run(self, radius: Union[int, float]):
        return float(radius) * 2.0 * pi

    def _arun(self, radius: int):
        raise NotImplementedError("This tool does not support async")


In [37]:
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent)
llm = ChatOpenAI(temperature=0.8, model_name="gpt-3.5-turbo")

# initialize conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)


In [38]:
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub


prompt = hub.pull("hwchase17/react")

tools = [CircumferenceTool()]

agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

prompt


PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [39]:
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

executor.invoke(
    input={
        "input": "can you calculate the circumference of a circle that has a radius of 7.81mm"
    }
)




> Entering new AgentExecutor chain...
Yes, I can calculate the circumference using the radius of the circle.
Action: Circumference calculator
Action Input: 7.8149.071677249072565I now know the circumference of a circle with a radius of 7.81mm.
Final Answer: The circumference of the circle is approximately 49.07mm.

> Finished chain.


{'input': 'can you calculate the circumference of a circle that has a radius of 7.81mm',
 'output': 'The circumference of the circle is approximately 49.07mm.'}